In [2]:
import os
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras import models, layers

2023-09-15 19:06:44.866786: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 19:06:45.037310: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 19:06:45.037905: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 19:06:46.257140: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
num_threads = 5
os.environ["OMP_NUM_THREADS"] = "5"
os.environ["TF_NUM_INTRAOP_THREADS"] = "5"
os.environ["TF_NUM_INTEROP_THREADS"] = "5"
tf.config.threading.set_inter_op_parallelism_threads(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)
tf.config.set_soft_device_placement(True)

In [14]:
MAX_WORDS_IN_SENTENCE = 500
WORD2VEC_DIMENSIONS = 100
CSV_PATH = './data/reviews-dataset.csv'

In [15]:
embeddings_index = {}
with open('/home/tina/Downloads/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [17]:
def clean_dataframe(df):
    cleaned_df = df.copy()
    pattern = r'[^a-zA-Z\s]'
    cleaned_df['text'] = cleaned_df['text'].apply(lambda x: re.sub(pattern, '', x))
    
    return cleaned_df

In [18]:
def text_to_word_vectors(text, vector_space_dimension):    
    sentence_vectors = []
    for word in text.lower().split():
        if word in embeddings_index and len(sentence_vectors) < MAX_WORDS_IN_SENTENCE:
            sentence_vectors.append(embeddings_index[word])    
    if len(sentence_vectors) > 0:
        pad_len = vector_space_dimension-len(sentence_vectors)      
        if pad_len > 0:
            padding = [np.zeros_like(sentence_vectors[0])] * pad_len
            sentence_vectors = sentence_vectors + padding
    return sentence_vectors

In [19]:
dataset = pd.read_csv(CSV_PATH)
dataset = clean_dataframe(dataset)
texts = dataset['text'].astype(str)
dataset['vlen'] = [len(text_to_word_vectors(t, MAX_WORDS_IN_SENTENCE)) for t in texts]
dataset = dataset[dataset.vlen > 0]

texts = dataset['text'].astype(str)
labels = dataset['label']

In [20]:
data = [text_to_word_vectors(text, MAX_WORDS_IN_SENTENCE) for text in texts]
data = np.array(data)
targets = np.array(labels).astype("float32")

In [21]:
SIZE = 1500
test_x = data[:SIZE]
test_x = np.array(test_x).astype('float32')
test_x =  test_x.reshape(len(test_x), MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS)
test_y = targets[:SIZE]

train_x = data[-SIZE:]
train_x = np.array(train_x).astype('float32')
train_x = train_x.reshape(len(train_x), MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS)
train_y = targets[-SIZE:]

In [22]:
model = models.Sequential()
model.add(layers.Dense(50, activation="relu", input_shape=(MAX_WORDS_IN_SENTENCE * WORD2VEC_DIMENSIONS, )))  
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid")) 
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 50)                2500050   
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense_5 (Dense)             (None, 50)                2550      
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_6 (Dense)             (None, 50)                2550      
                                                                 
 dense_7 (Dense)             (None, 1)                 51        
                                                                 
Total params: 2505201 (9.56 MB)
Trainable params: 2505

In [23]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
results = model.fit(train_x, train_y, epochs=20, batch_size=4, validation_data=(test_x, test_y))

Epoch 1/20


2023-09-15 19:11:04.808333: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 300000000 exceeds 10% of free system memory.


374/375 [============================>.] - ETA: 0s - loss: 0.7804 - accuracy: 0.5033

2023-09-15 19:11:16.116525: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 300000000 exceeds 10% of free system memory.


375/375 [==============================] - 16s 41ms/step - loss: 0.7804 - accuracy: 0.5027 - val_loss: 0.6928 - val_accuracy: 0.5173
Epoch 2/20
375/375 [==============================] - 12s 31ms/step - loss: 0.7091 - accuracy: 0.4860 - val_loss: 0.6932 - val_accuracy: 0.4893
Epoch 3/20
375/375 [==============================] - 11s 30ms/step - loss: 0.7011 - accuracy: 0.5007 - val_loss: 0.6934 - val_accuracy: 0.4947
Epoch 4/20
375/375 [==============================] - 11s 29ms/step - loss: 0.6964 - accuracy: 0.5113 - val_loss: 0.6932 - val_accuracy: 0.4973
Epoch 5/20
375/375 [==============================] - 11s 28ms/step - loss: 0.6923 - accuracy: 0.5240 - val_loss: 0.6920 - val_accuracy: 0.5193
Epoch 6/20
375/375 [==============================] - 11s 29ms/step - loss: 0.6786 - accuracy: 0.5480 - val_loss: 0.6944 - val_accuracy: 0.5060
Epoch 7/20
375/375 [==============================] - 11s 28ms/step - loss: 0.6658 - accuracy: 0.5820 - val_loss: 0.6921 - val_accuracy: 0.5307
Epo

In [13]:
model.save('./data/keras-review-300d.mdl')
loaded_model = models.load_model('./data/keras-review-300d.mdl')
scores = loaded_model.evaluate(test_x, test_y, verbose=0)
print("Loaded Model Accuracy: %.2f%%" % (scores[1] * 100))

INFO:tensorflow:Assets written to: ./data/keras-review-300d.mdl/assets


INFO:tensorflow:Assets written to: ./data/keras-review-300d.mdl/assets
2023-09-15 19:10:00.387830: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 900000000 exceeds 10% of free system memory.


Loaded Model Accuracy: 53.60%


In [ ]:
scores = loaded_model.evaluate(test_x, test_y, verbose=0)
print("Loaded Model Accuracy: %.2f%%" % (scores[1] * 100))

In [28]:
pd.set_option("max_colwidth", 1000)
dataset.head()

,text,label,vlen
0,good question Nepal PM declaring to resign and actually resigning is two very different things,1,500
1,i just saw your performance on the ellen show were so behind in australia you were AMAZING and have such a wonderful voice,1,500
2,nooo as a font connoisseur i can totally relate dude my heart goes out to you,0,500
3,i really am going to miss you,0,500
4,Ahhh Im engulfed in shooooes Who told me to buy so maany shoes,0,500


2948